In [46]:
import pymongo
import pandas as pd
import numpy as np
from sklearn import datasets
import pickle
import time
import pymongo
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
import random
from pymongo import MongoClient
import pandas as pd
import os

# Loading Model 

In [2]:
def load_saved_model_from_db(model_name, client, db, dbconnection):
    json_data = {}
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    data = mycon.find({'name': model_name})
    
    
    for i in data:
        json_data = i
    #fetching model from db
    pickled_model = json_data[model_name]
    
    return pickle.loads(pickled_model)

In [3]:
details = {
        'inserted_id':"ObjectId('5eb3e0954668994dc15066e2')",
        'model_name':"CNN",
        'created_time':1588846741.541345 }

In [4]:
model  = load_saved_model_from_db(model_name = details['model_name'], client = 'mongodb://localhost:27017/', 
                         db = 'Cat_dog_classifier', dbconnection = 'Model_Accuracy_95.0%')

In [5]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=[IMAGE_WIDTH, IMAGE_HEIGHT]
IMAGE_CHANNELS=3
batch_size=15

# Test Data 

In [6]:
test_filenames = os.listdir("test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [7]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

Found 12500 validated image filenames.


In [8]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [9]:
test_df['category'] = np.argmax(predict, axis=-1)

In [10]:
test_df

,filename,category
0,11835.jpg,1
1,4737.jpg,1
2,3323.jpg,1
3,4758.jpg,0
4,5678.jpg,0
...,...,...
12495,6422.jpg,1
12496,3040.jpg,0
12497,10816.jpg,0
12498,2124.jpg,1


In [15]:
lables = {('cat', 0), ('dog', 1)}

In [16]:
label_map = dict((v,k) for k,v in lables)
test_df['category'] = test_df['category'].replace(label_map)

In [17]:
test_df

,filename,category
0,11835.jpg,dog
1,4737.jpg,dog
2,3323.jpg,dog
3,4758.jpg,cat
4,5678.jpg,cat
...,...,...
12495,6422.jpg,dog
12496,3040.jpg,cat
12497,10816.jpg,cat
12498,2124.jpg,dog


In [38]:
pro = []
for index, probability in enumerate(predict):
    probability = probability*100
    pro.append(probability)
    
prob = pd.DataFrame(pro)
prob.columns = ['Dog','Cat']
prob.columns

Index(['Dog', 'Cat'], dtype='object')

In [60]:
def save(x):
    if x['Dog'] >= x['Cat']:
        return x['Dog']
    else:
        return x['Cat']
prob['Prediction'] = prob.apply(save, axis=1)

In [59]:
prob

,Dog,Cat,Prediction
0,0.00000,1.000000e+02,100.000000
1,0.00000,1.000000e+02,100.000000
2,0.00000,1.000000e+02,100.000000
3,100.00000,0.000000e+00,100.000000
4,100.00000,0.000000e+00,100.000000
...,...,...,...
12495,0.00000,1.000000e+02,100.000000
12496,100.00000,0.000000e+00,100.000000
12497,100.00000,0.000000e+00,100.000000
12498,40.03252,5.996748e+01,59.967484


In [43]:
Prediction = pd.concat([test_df,prob],axis=1)

In [44]:
Prediction

,filename,category,Dog,Cat,Prediction
0,11835.jpg,dog,0.00000,1.000000e+02,100.000000
1,4737.jpg,dog,0.00000,1.000000e+02,100.000000
2,3323.jpg,dog,0.00000,1.000000e+02,100.000000
3,4758.jpg,cat,100.00000,0.000000e+00,100.000000
4,5678.jpg,cat,100.00000,0.000000e+00,100.000000
...,...,...,...,...,...
12495,6422.jpg,dog,0.00000,1.000000e+02,100.000000
12496,3040.jpg,cat,100.00000,0.000000e+00,100.000000
12497,10816.jpg,cat,100.00000,0.000000e+00,100.000000
12498,2124.jpg,dog,40.03252,5.996748e+01,59.967484


In [51]:
client = MongoClient('localhost', 27017)
db=client.Cat_dog_classifier
Model_Accuracy_95_prediction = db.Model_Accuracy_95_prediction

In [53]:
records_ = Prediction.to_dict(orient = 'records')
result = db.Model_Accuracy_95_prediction.insert_many(records_ )